In [1]:
import json 

# retrieve password and connection string
with open('/Users/vanes/ThisPC/Documents/GitHub/credentials.json') as f: 
    data = json.load(f)
    mongo_conn_str = data['mongodb']

In [22]:
import pymongo

# connect to mongodb and database
client = pymongo.MongoClient(mongo_conn_str)
db = client.admin

import re 
result = client['Chirps']['Chirps'].aggregate([
    {
        'addFields': {
            'sentiment': {
                '$switch': {
                    'branches': [
                        {
                            'case': {
                                '$regexMatch': {
                                    'input': '$comment', 
                                    'regex': (r"I really hate")
                                }
                            },
                            'then': 1
                        }, { 
                            'case': {
                                '$regexMatch': { 
                                    'input': '$comment', 
                                    'regex': (r"I hate")
                                }
                            },
                            'then': 2 
                        }, { 
                            'case': { 
                                '$regexMatch' : { 
                                    'input' : '$comment', 
                                    'regex' : (r"Maybe I like")
                                }
                            }, 
                            'then': 3
                        }, { 
                            'case': {
                                '$regexMatch' : {
                                    'input': '$comment', 
                                    'regex': (r"I think I might like")
                                }
                            }, 
                            'then': 3 
                        }, { 
                            'case': { 
                                '$regexMatch': { 
                                    'input': '$comment', 
                                    'regex': (r"I like") 
                                } 
                            }, 
                            'then': 4
                        }, { 
                            'case': { 
                                '$regexMatch': { 
                                    'input': '$comment', 
                                    'regex': (r"I love") 
                                }
                            }, 
                            'then': 5
                        }
                    ], 
                    'default': 0
                } 
            } 
        } 
    }, { 
        '$match': { 
            'comment': re.compile(r"hiking"), 
            'likes': { 
                '$gt': 10 
            } 
        } 
    }, { 
        '$addFields': { 
            'year': { 
                '$substr': [ 
                    '$date', 0, 4 
                ] 
            }, 
            'month': { 
                '$substr': [ 
                    '$date', 5, 2 
                ] 
            }, 
            'day': { 
                '$substr': [ 
                    '$date', 8, 2
                ] 
            }
        } 
    }, { 
        '$group': { 
            '_id': { 
                'year': '$year', 
                'month': '$month'
            }, 
            'total_chirps': { 
            '$sum': 1 
            }, 
            'average_sentiment': { 
            '$avg': '$sentiment' 
            }
        }
    }
]) 

for i in result: 
    print(i) 
        

OperationFailure: addFields is not allowed in this atlas tier, full error: {'ok': 0, 'errmsg': 'addFields is not allowed in this atlas tier', 'code': 8000, 'codeName': 'AtlasError'}